In [2]:
import numpy as np
import pandas as pd
from hpobench.benchmarks.ml.tabular_benchmark import TabularBenchmark
from ConfigSpace.hyperparameters import OrdinalHyperparameter
from ConfigSpace.configuration_space import ConfigurationSpace

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [3]:
benchmark = TabularBenchmark('nn', 31)

In [14]:
import torch
from typing import Dict

def find_nearest_ordinal(value: float, hyperparameter_type: OrdinalHyperparameter):
    # Convert the sequence to a PyTorch tensor
    valid_values = torch.tensor(hyperparameter_type.sequence)
    
    # Calculate the nearest value using torch operations
    nearest = torch.argmin((valid_values - value) ** 2).item()
    order = hyperparameter_type.get_seq_order()

    return hyperparameter_type.get_value(order[nearest])

def round_to_valid_config(values: Dict[str, float], space: ConfigurationSpace):
    # Iterate over hyperparameters using a dictionary comprehension
    return {hyperparameter.name: find_nearest_ordinal(values[hyperparameter.name], hyperparameter) for hyperparameter in space.get_hyperparameters()}

def modified_objective_function(values: torch.Tensor):
    results = []
    for i in range(values.size(0)):  # Looping over each point
        config = {
            "alpha": 10 ** ((values[i, 0].detach().item() - 1) * 8),
            "batch_size": 2 ** (values[i, 1].detach().item() * 8),
            "depth": int(2 * values[i, 2].detach().item() + 1),  # Ensuring depth is an integer
            "learning_rate_init": 10 ** ((values[i, 3].detach().item() - 1) * 5),
            "width": 2 ** (values[i, 4].detach().item() * 10)
        }

        result = benchmark.objective_function(round_to_valid_config(config, benchmark.configuration_space))
        results.append(result['function_value'])

    return torch.tensor(results)  # Convert the list of results to an N*1 tensor

In [33]:
benchmark.get_global_min('val')['acc']

0.08956228956228955